In [3]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sklearn

In [20]:
patient_df = pd.read_csv('data/allpatients_imputed_df').drop(columns=['Unnamed: 0'])

In [21]:
patient_df.shape

(188453, 42)

In [22]:
patient_df.columns

Index(['pid', 'HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'EtCO2',
       'BaseExcess', 'HCO3', 'FiO2', 'pH', 'PaCO2', 'SaO2', 'AST', 'BUN',
       'Alkalinephos', 'Calcium', 'Chloride', 'Creatinine', 'Bilirubin_direct',
       'Glucose', 'Lactate', 'Magnesium', 'Phosphate', 'Potassium',
       'Bilirubin_total', 'TroponinI', 'Hct', 'Hgb', 'PTT', 'WBC',
       'Fibrinogen', 'Platelets', 'Age', 'Gender', 'Unit1', 'Unit2',
       'HospAdmTime', 'ICULOS', 'SepsisLabel'],
      dtype='object')

#### Fill in NaN values with mean of the column

In [24]:
patient_df = patient_df.fillna(patient_df.mean()['HR':'Platelets'])

#### Sample 15 time-steps from each patient to create new dataset X

In [42]:
sepsis_patients = patient_df.groupby('pid').filter(lambda x: x['SepsisLabel'].any())
normal_patients = patient_df.groupby('pid').filter(lambda x: x['SepsisLabel'].sum() == 0)

In [77]:
(sepsis_patients.groupby('pid').ICULOS.agg(['count'])['count'] < 13).sum()

73

In [165]:
sepsis_patient_agg = sepsis_patients.groupby('pid')
X_sepsis = pd.DataFrame(data=[])

for pid, data in sepsis_patient_agg:
    data.reset_index(drop=True, inplace=True)
    
    if data.shape[0] >= 13:
        first_sepsis_idx =  data['SepsisLabel'].idxmax()
        start_idx = first_sepsis_idx - 6
        
        if start_idx >= 0:
            new_data = data.iloc[start_idx:first_sepsis_idx]
            assert new_data.shape[0] == 6
            X_sepsis = pd.concat([X_sepsis, new_data])

In [166]:
X_sepsis.shape

(1176, 42)

In [167]:
normal_patient_agg = normal_patients.groupby('pid')
X_normal = pd.DataFrame(data=[])

for pid, data in normal_patient_agg:
    data.reset_index(drop=True, inplace=True)
    new_data = data.sample(n=6)
    X_normal = pd.concat([X_normal, new_data])

In [79]:
X_normal.pid.unique().shape

(4721,)

In [170]:
X_sepsis[X_sepsis.pid == 'p00049']

,pid,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,...,WBC,Fibrinogen,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel
72,p00049,68.0,98.0,37.4,134.0,104.0,82.0,13.0,33.517515,-3.160918,...,20.4,296.932741,338.0,60,0,NaN,NaN,-0.03,73,0
73,p00049,72.0,99.0,37.6,168.0,114.0,80.0,22.0,33.517515,-3.160918,...,20.4,296.932741,338.0,60,0,NaN,NaN,-0.03,74,0
74,p00049,72.0,97.0,37.6,154.0,108.0,80.0,21.0,33.517515,-3.160918,...,20.4,296.932741,338.0,60,0,NaN,NaN,-0.03,75,0
75,p00049,100.0,98.0,37.6,146.0,110.0,84.0,20.0,33.517515,-3.160918,...,20.4,296.932741,338.0,60,0,NaN,NaN,-0.03,76,0
76,p00049,71.5,98.5,37.6,162.0,107.0,63.5,20.0,33.517515,-3.160918,...,20.4,296.932741,338.0,60,0,NaN,NaN,-0.03,77,0
77,p00049,52.0,98.0,39.1,114.0,84.0,60.0,21.0,33.517515,-3.160918,...,20.4,296.932741,338.0,60,0,NaN,NaN,-0.03,78,0


In [178]:
X = np.ndarray((X_sepsis.pid.unique().shape[0] + X_normal.pid.unique().shape[0], 228))
y = [1]*(X_sepsis.pid.unique().shape[0]) + [0]*(X_normal.pid.unique().shape[0])

i = 0
for pid, df in X_sepsis.groupby('pid'):
    df = df.drop(columns=['pid', 'Unit1', 'Unit2', 'SepsisLabel'])
    x_i = df.stack().values
    X[i,:] = x_i
    i += 1

for pid, df in X_normal.groupby('pid'):
    df = df.drop(columns=['pid', 'Unit1', 'Unit2', 'SepsisLabel'])
    x_i = df.stack().values
    X[i,:] = x_i
    i += 1

In [179]:
X.shape

(4917, 228)

In [181]:
y = np.array(y)

In [182]:
y.shape

(4917,)

## LR, SVM, and RF Baseline Models

In [123]:
np.array([[1,2],[3,4]])

array([[1, 2],
       [3, 4]])